In [ ]:
!pip install sacrerouge

In [ ]:
!pip install urllib3==1.25.10

In [ ]:
!apt install libxml-dom-perl
!sacrerouge setup-metric rouge

In [4]:
import pickle
import pandas as pd
import numpy as np

**Load WMT15 data**

In [13]:
data_preproc = 'tokenized_and_lowercase'

In [12]:
data_type = 'wmt15'
with open("/content/mt.txt") as f:
    hyp_snts = [line[:-1] for line in f] # remove last symbol which is '\n'

with open("/content/reference.txt") as f:
    ref_snts = [line[:-1] for line in f]

with open("/content/newstest2015.human.de-en") as f:
    human_scores = [float(line[:-1]) for line in f] 

**Load WMT21 data**

In [ ]:
data_type = 'wmt21'
with open('/content/all_ref_snts_21.pickle', 'rb') as fp:
    ref_snts = pickle.load(fp)

with open('/content/all_mt_snts_21.pickle', 'rb') as fp:
    hyp_snts = pickle.load(fp)

with open('/content/all_src_snts_21.pickle', 'rb') as fp:
    src_snts = pickle.load(fp)

In [ ]:
# check the ROUGE output format

from sacrerouge.metrics import Rouge
metric = Rouge(compute_rouge_l=True, remove_stopwords=False) #  remove_stopwords = True (False by default and performs better)

scores = metric.score(hyp_snts[0], [ref_snts[0]])
scores

In [35]:
# run ROUGE on all data

rouge1_f1 = []
rouge1_p = []
rouge1_r = []
rougel_f1 = []

# The score function expects a list of reference summaries, so `reference` is
# wrapped in a list
for i, reference in enumerate(ref_snts):
    scores = metric.score(hyp_snts[i], [reference])
    rouge1_f1.append(scores['rouge-1']['f1'])
    rouge1_p.append(scores['rouge-1']['precision'])
    rouge1_r.append(scores['rouge-1']['recall'])
    rougel_f1.append(scores['rouge-l']['f1'])

In [36]:
all_rouge_scores = {}
all_rouge_scores["rouge1_f1"] = rouge1_f1
all_rouge_scores["rouge1_p"] = rouge1_p
all_rouge_scores["rouge1_r"] = rouge1_r
all_rouge_scores["rougel_f1"] = rougel_f1

In [37]:
scores_dict = {'human': human_scores}
scores_dict.update(all_rouge_scores)
metric_scores = pd.DataFrame(scores_dict)

In [38]:
# stopwords removed
metric_scores.corr()

,human,rouge1_f1,rouge1_p,rouge1_r,rougel_f1
human,1.000000,0.451041,0.408538,0.463725,0.488308
rouge1_f1,0.451041,1.000000,0.971387,0.975379,0.934573
rouge1_p,0.408538,0.971387,1.000000,0.898899,0.900715
rouge1_r,0.463725,0.975379,0.898899,1.000000,0.919439
rougel_f1,0.488308,0.934573,0.900715,0.919439,1.000000


In [27]:
# stopwords are kept
metric_scores.corr()

,human,rouge1_f1,rouge1_p,rouge1_r,rougel_f1
human,1.000000,0.583409,0.542574,0.571295,0.613401
rouge1_f1,0.583409,1.000000,0.948851,0.955887,0.907558
rouge1_p,0.542574,0.948851,1.000000,0.818325,0.862849
rouge1_r,0.571295,0.955887,0.818325,1.000000,0.867353
rougel_f1,0.613401,0.907558,0.862849,0.867353,1.000000


In [30]:
with open(f'all_rouge_scores_{data_type}_{data_preproc}.pickle', 'wb') as f:
    pickle.dump(all_rouge_scores, f, pickle.HIGHEST_PROTOCOL)